In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
import time

In [3]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].str.replace('.', '-')

In [4]:
def remove_last_col(df):
    if len(df.columns) > N:
        df = df.iloc[:, :-1]
    return df

In [26]:
yearly_combined = pd.DataFrame()
N = 4
# tickers[0]
for ticker in ['NVDA']:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        balance = stock.balance_sheet
        balance = remove_last_col(balance)

        financials = stock.financials
        financials = remove_last_col(financials)

        earnings = stock.income_stmt
        earnings = remove_last_col(earnings)

        cashflow = stock.cash_flow
        cashflow = remove_last_col(cashflow)

        start_dates = balance.columns.values
        end_dates = start_dates + np.timedelta64(366, 'D')

        start_dates = [str(date)[: -19] for date in start_dates]
        end_dates = [str(date)[: -19] for date in end_dates]
        

        year1 = stock.history(start=start_dates[0], end=end_dates[0]).reset_index(drop=True)
        year2 = stock.history(start=start_dates[1], end=end_dates[1]).reset_index(drop=True)
        year3 = stock.history(start=start_dates[2], end=end_dates[2]).reset_index(drop=True)
        year4 = stock.history(start=start_dates[3], end=end_dates[3]).reset_index(drop=True)
        print(start_dates)
        print(end_dates)

        columns = balance.columns.values
        print(columns)
        balance.columns = columns
        earnings.columns= columns
        cashflow.columns = columns
        financials.columns = columns

        yearly = pd.concat([balance, earnings, cashflow, financials]).T
        columns = yearly.columns.values
        #print('jello')
        #for col in columns:
        #    print(col)
        #yearly['gross_margin'] = yearly['Gross Profit'] / yearly['Total Revenue']
        print(yearly['Gross Profit'])
        print(yearly['Total Revenue'])
        print(yearly['Gross Profit'] / yearly['Total Revenue'])
        # Liquidity
        # current ratio = current assets / current liabilities


        yearly['current_ratio'] = yearly['Current Assets'] / yearly['Current Liabilities']
        # quick ratio = Inventory + current assets / current liabilities
        yearly['quick_ratio'] = yearly['Total Assets']
        if 'Inventory' in columns:
            yearly['quick_ratio'] += yearly['Inventory']
        yearly['quick_ratio'] /= yearly['Total Liabilities Net Minority Interest']
        # cash ratio = (cash + cash equivalents) / current liabilities
        yearly['cash_ratio'] = yearly['Cash And Cash Equivalents'] / yearly['Total Liabilities Net Minority Interest']
        yearly['operating_cashflow'] = yearly['Operating Cash Flow']
        # equity ratio = total equity / total assets
        yearly['equity_ratio'] = yearly['Stockholders Equity'] / yearly['Total Assets']

        # Leverage
        # debt to equity = total liabilites / shareholder equity
        yearly['debt_to_equity'] = yearly['Total Liabilities Net Minority Interest'] / yearly['Stockholders Equity']
        # debt to assets = total liabilites / shareholder equity
        yearly['debt_to_assets'] = yearly['Total Liabilities Net Minority Interest'] / yearly['Total Assets']
        # interest coverage = operating income / interest expense
        yearly['interest_coverage'] = yearly['Operating Revenue'] / yearly['Interest Expense']

        # Efficiency
        # asset turnover = revenue / total assets
        yearly['asset_turnover'] = yearly['Total Revenue'] / yearly['Total Assets']
        # receivables turnover = revenue / accounts receivables
        if 'Receivables' in columns: yearly['receivables_turnover'] = yearly['Total Revenue'] / yearly['Receivables']
        else: yearly['receivables_turnover'] = np.nan

        # Profitability
        # gross margin = gross profit / revenue
        yearly['gross_margin'] = yearly['Gross Profit'] / yearly['Total Revenue']
        # profit margin = net income / revenue
        yearly['profit_margin'] = yearly['Net Income'] / yearly['Total Revenue']
        # return on equity = net income / shareholder equity
        yearly['return_on_equity'] = yearly['Net Income'] / yearly['Stockholders Equity']
        # return on assets = net income / total assets
        yearly['return_on_assets'] = yearly['Net Income'] / yearly['Total Assets']
        # eps

        shareprice = np.array([year1.loc[0, 'Close'], year2.loc[0, 'Close'], year3.loc[0, 'Close'], year4.loc[0, 'Close']])
        # p/e = shareprice / eps
        yearly['p/e'] = shareprice / yearly['eps']

        yearly['ticker'] = ticker
        yearly['sector'] = info['sector']
        yearly.index = yearly.index.year
        yearly['year'] = yearly.index.values
        
        startprice = np.array([year1.loc[1, 'Open'], year2.loc[1, 'Open'], year3.loc[1, 'Open'], year4.loc[1, 'Open']])
        endprice = np.array([year1.loc[len(year1)-1, 'Close'], year2.loc[len(year2)-1, 'Close'], year3.loc[len(year3)-1, 'Close'], year4.loc[len(year4)-1, 'Close']])
        pctdiff = (endprice-startprice) / startprice
        yearly['result'] = pctdiff
                
        columns = ["current_ratio", "quick_ratio", "cash_ratio", "operating_cashflow", "equity_ratio", "debt_to_equity", "debt_to_assets", "interest_coverage", "asset_turnover", "receivables_turnover", "gross_margin", "profit_margin", "return_on_equity", "return_on_assets", "eps", "p/e", "ticker", "sector", "year", "result"]

        yearly = yearly[columns].reset_index(drop=True)
        
        if yearly_combined.empty:
            yearly_combined = yearly
        else:
            yearly_combined = pd.concat([yearly_combined, yearly]).reset_index(drop=True)
        print(ticker, 'is done')
        

    except Exception as e:
        print(ticker, e)

yearly_combined.to_csv('combined.csv', index=False)

['2024-01-31', '2023-01-31', '2022-01-31', '2021-01-31']
['2025-01-31', '2024-02-01', '2023-02-01', '2022-02-01']
['2024-01-31T00:00:00.000000000' '2023-01-31T00:00:00.000000000'
 '2022-01-31T00:00:00.000000000' '2021-01-31T00:00:00.000000000']
             Gross Profit   Gross Profit
2024-01-31  44301000000.0  44301000000.0
2023-01-31  15356000000.0  15356000000.0
2022-01-31  17475000000.0  17475000000.0
2021-01-31  10396000000.0  10396000000.0
            Total Revenue  Total Revenue
2024-01-31  60922000000.0  60922000000.0
2023-01-31  26974000000.0  26974000000.0
2022-01-31  26914000000.0  26914000000.0
2021-01-31  16675000000.0  16675000000.0
           Gross Profit Gross Profit Total Revenue Total Revenue
2024-01-31          NaN          NaN           NaN           NaN
2023-01-31          NaN          NaN           NaN           NaN
2022-01-31          NaN          NaN           NaN           NaN
2021-01-31          NaN          NaN           NaN           NaN
NVDA Cannot set a Da

In [22]:
10396000000.0 / 16675000000

0.623448275862069

[ 1.08142815  2.12601203 -0.22102979  0.82907985]
NVDA Cannot set a DataFrame with multiple columns to the single column interest_coverage
